<a href="https://colab.research.google.com/github/binghubli/Competition/blob/main/Savitsky_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy.optimize import minimize

# --- 1. 国际制 (SI) 输入参数定义 ---

# 参数来源于原始图片值转换：
# 船体几何和操作条件
Delta_input = 266893.2  # N (牛顿，来自 60000 LB)
LCG = 8.8392          # m (米，来自 29.0 FT)
b = 4.2672            # m (米，来自 14.0 FT)
V = 20.574            # m/s (米/秒，来自 67.5 ft/s)
f = 0.1524            # m (米，推力线高度，来自 0.50 FT)
Beta_deg = 10.0       # 度 (deg, 斜升角)
epsilon_deg = 4.0     # 度 (deg, 推力线角)

# 流体与环境常数
g = 9.81              # m/s^2 (重力加速度)
rho = 1019.14         # kg/m^3 (密度，海水)
nu = 1.18e-6          # m^2/s (运动粘度)


# --- 2. 定义平衡方程组 ---
def savitsky_equations(p, Delta_input, LCG, b, Beta_deg, V, f, epsilon_deg, rho, g, nu):
    tau_deg, Lambda = p

    # 物理约束检查 (防止求解器进入非物理区域)
    if tau_deg <= 0.5 or Lambda <= 1.0:
        return [1e10, 1e10]

    tau_rad = np.deg2rad(tau_deg)

    # 升力平衡计算 (Delta_calc - Delta_input = 0)
    Cv = V / np.sqrt(g * b)
    CL0_ratio = 0.0120 * np.sqrt(Lambda) + (0.0055 * Lambda**2.5) / (Cv**2)
    CL0 = CL0_ratio * (tau_deg**1.1)
    CL_Beta = CL0 - 0.0065 * Beta_deg * (CL0**0.6)
    Delta_calc = CL_Beta * (0.5 * rho * V**2 * b**2)
    Lift_Error = Delta_calc - Delta_input

    # 力矩平衡计算 (M_Hydro - M_Thrust = 0)
    Re = V * b * Lambda / nu
    if Re <= 0: return [1e10, 1e10]

    Cf = 0.075 / (np.log10(Re) - 2)**2
    dCf = 0.0004
    S = Lambda * b**2 / np.cos(np.deg2rad(Beta_deg))
    Df = 0.5 * rho * V**2 * S * (Cf + dCf)
    D = Delta_input * np.tan(tau_rad) + Df / np.cos(tau_rad)

    Chi = Cv / Lambda
    Cp = 0.75 - 1 / (5.21 * (Chi**2) + 2.39)

    a = LCG - b / 4 * np.tan(np.deg2rad(Beta_deg))
    M_Hydro = D * (a - f)
    M_Thrust = Delta_input * f * np.sin(tau_rad + np.deg2rad(epsilon_deg))

    Moment_Error = M_Hydro - M_Thrust

    return [Lift_Error, Moment_Error]


# --- 3. 定义目标函数 (最小化误差平方和) ---
def objective_function(p, *args):
    Delta_input, LCG, b, *rest = args
    Lift_Error, Moment_Error = savitsky_equations(p, *args)
    Lift_Error_norm = Lift_Error / Delta_input
    Moment_Error_norm = Moment_Error / (Delta_input * b)
    return Lift_Error_norm**2 + Moment_Error_norm**2


# --- 4. 求解与结果输出 ---
initial_guess = [3.5, 1.9]
args_tuple = (Delta_input, LCG, b, Beta_deg, V, f, epsilon_deg, rho, g, nu)

print("--- 正在求解 Savitsky 平衡状态 (SI 单位制) ---")

result = minimize(
    objective_function,
    initial_guess,
    args=args_tuple,
    method='Nelder-Mead',
    options={'maxiter': 500}
)

if result.success or result.fun < 1e-6:
    tau_solved, lambda_solved = result.x

    # 重新计算最终阻力 D 和功率 P_E
    def calculate_final_results(tau, Lambda):
        tau_rad = np.deg2rad(tau)
        S = Lambda * b**2 / np.cos(np.deg2rad(Beta_deg))

        # 阻力项需要重新计算 Re, Cf, Df
        Cv = V / np.sqrt(g * b)
        Re = V * b * Lambda / nu
        Cf = 0.075 / (np.log10(Re) - 2)**2
        dCf = 0.0004
        Df = 0.5 * rho * V**2 * S * (Cf + dCf)
        D_Total = Delta_input * np.tan(tau_rad) + Df / np.cos(tau_rad)

        Power_W = D_Total * V
        Power_kW = Power_W / 1000.0

        return D_Total, Power_kW

    R_Total_N, Power_kW = calculate_final_results(tau_solved, lambda_solved)
    final_errors = savitsky_equations(result.x, *args_tuple)

    print("\n--- 优化求解结果 (SI 单位制) ---")
    print(f"纵倾角 (tau): {tau_solved:.3f}°")
    print(f"浸湿长度比 (lambda): {lambda_solved:.3f}")

    print("\n--- 最终输出 ---")
    print(f"总阻力 (D): {R_Total_N:,.0f} N")
    print(f"有效功率 (P_E): {Power_kW:.1f} kW")

    print("\n--- 误差检查 ---")
    print(f"最终最小化误差平方和: {result.fun:.2e}")
    print(f"升力误差检查: {final_errors[0]:.2e} N")
    print(f"力矩误差检查: {final_errors[1]:.2e} N·m")

else:
    print("\n--- 求解失败 ---")
    print(f"失败原因: {result.message}")

--- 正在求解 Savitsky 平衡状态 (SI 单位制) ---

--- 优化求解结果 (SI 单位制) ---
纵倾角 (tau): 4.402°
浸湿长度比 (lambda): 1.428

--- 最终输出 ---
总阻力 (D): 34,626 N
有效功率 (P_E): 712.4 kW

--- 误差检查 ---
最终最小化误差平方和: 6.52e-02
升力误差检查: -8.88e+03 N
力矩误差检查: 2.88e+05 N·m
